In [2]:
import cv2
import supervision as sv
from inference import get_model
from PIL import Image

In [3]:
def showNumpyImage(image):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_image)
    pil_image.show()

In [7]:
# 객체를 인식하고, 사각형 박스 형태로 인식된 객체에 대한 라벨을 붙이는 예시
model = get_model(model_id="yolov10n-640")
image = cv2.imread("./rawDatas/people-walking-2.webp")
results = model.infer(image)[0]

detections = sv.Detections.from_inference(results)
annotated_image = sv.BoundingBoxAnnotator().annotate(
    scene=image, detections=detections)

labels = [
    f"{class_name} {confidence:.2f}"
    for class_name, confidence
    in zip(detections['class_name'], detections.confidence)
]
annotated_image = sv.LabelAnnotator().annotate(
    scene=annotated_image, detections=detections, labels=labels)

showNumpyImage(annotated_image)

In [11]:
# 객체를 인식하고, Segnantation 형태로 인식된 객체에 대한 라벨을 붙이는 예시
model = get_model(model_id="yolov8n-seg-640")
image = cv2.imread("./rawDatas/people-walking-2.webp")
results = model.infer(image)[0]

detections = sv.Detections.from_inference(results)
annotated_image = sv.MaskAnnotator().annotate(
    scene=image, detections=detections)

annotated_image = sv.LabelAnnotator().annotate(
    scene=annotated_image, detections=detections, labels=labels)

showNumpyImage(annotated_image)

In [13]:
# 비디오 객체 탐지 관련 정보를 CSV로 저장시키기
model = get_model(model_id="yolov10n-640")
frames_generator = sv.get_video_frames_generator("./rawDatas/people-walking.mp4")

with sv.CSVSink("./outputs/detections.csv") as sink:
    for frame in frames_generator:
        results = model.infer(frame)[0]
        detections = sv.Detections.from_inference(results)
        sink.append(detections, {})